In [1]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import pandas as pd
import time
import os
from datetime import datetime, timedelta

In [7]:
def adjust_to_market_day(pub_datetime):
    # Eğer saat 16:00'dan sonra ise -> bir gün ileri al
    if pub_datetime.hour >= 16:
        pub_datetime += timedelta(days=1)

    # Eğer çıkan tarih hafta sonuna denk gelirse, Pazartesi’ye kaydır
    while pub_datetime.weekday() > 4:  # 5 = Cumartesi, 6 = Pazar
        pub_datetime += timedelta(days=1)

    return pub_datetime.date()

In [8]:
def fetch_nyt_articles(year, month, api_key):
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"{year}-{month:02} verisi çekilemedi: {response.status_code}")
        return pd.DataFrame()

    docs = response.json()["response"]["docs"]

    records = []
    for doc in docs:
        pub_date_str = doc.get("pub_date", "")
        try:
            pub_date = pd.to_datetime(pub_date_str).tz_convert("US/Eastern")
        except:
            continue

        aligned_date = adjust_to_market_day(pub_date)
        
        records.append({
            "aligned_date": aligned_date,
            "headline": doc.get("headline", {}).get("main", ""),
            "text": doc.get("lead_paragraph") or doc.get("abstract") or "",
            "section": doc.get("section_name", ""),
            "url": doc.get("web_url", "")
        })

    return pd.DataFrame(records)

In [9]:
# Çıktı klasörü
output_folder = "data/news/nyt_aligned"
os.makedirs(output_folder, exist_ok=True)

api_key = "IiGdGTjvrJEaUHkre4cMfKwde8mq2n9R"

for year in range(2004, 2025):
    for month in range(1, 13):
        print(f"{year}-{month:02} işleniyor...")
        df = fetch_nyt_articles(year, month, api_key)

        if not df.empty:
            df.to_csv(f"{output_folder}/nyt_{year}_{month:02}.csv", index=False)

        time.sleep(6)  # NYT API rate limit

2004-01 işleniyor...
2004-02 işleniyor...
2004-03 işleniyor...
2004-04 işleniyor...
2004-05 işleniyor...
2004-06 işleniyor...
2004-07 işleniyor...
2004-08 işleniyor...
2004-09 işleniyor...
2004-10 işleniyor...
2004-11 işleniyor...
2004-12 işleniyor...
2005-01 işleniyor...
2005-02 işleniyor...
2005-03 işleniyor...
2005-04 işleniyor...
2005-05 işleniyor...
2005-06 işleniyor...
2005-07 işleniyor...
2005-08 işleniyor...
2005-09 işleniyor...
2005-10 işleniyor...
2005-11 işleniyor...
2005-12 işleniyor...
2006-01 işleniyor...
2006-02 işleniyor...
2006-03 işleniyor...
2006-04 işleniyor...
2006-05 işleniyor...
2006-06 işleniyor...
2006-07 işleniyor...
2006-08 işleniyor...
2006-09 işleniyor...
2006-10 işleniyor...
2006-11 işleniyor...
2006-12 işleniyor...
2007-01 işleniyor...
2007-02 işleniyor...
2007-03 işleniyor...
2007-04 işleniyor...
2007-05 işleniyor...
2007-06 işleniyor...
2007-07 işleniyor...
2007-08 işleniyor...
2007-09 işleniyor...
2007-10 işleniyor...
2007-11 işleniyor...
2007-12 işlen

In [12]:
import glob

files = glob.glob("data/news/nyt_aligned/*.csv")
df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
df_all["date"] = pd.to_datetime(df_all["aligned_date"])

In [14]:
relevant_sections = ["Business Day", "U.S.", "World", "Opinion", "Technology",
    "Economy", "Your Money", "Markets", "Briefing", "Washington"]
keywords = "market|stock|nasdaq|s&p|inflation|fed|oil|recession|bond|finance|interest"

df_filtered = df_all[
    (df_all["section"].isin(relevant_sections)) |
    (df_all["headline"].str.contains(keywords, case=False, na=False)) |
    (df_all["text"].str.contains(keywords, case=False, na=False))
]

In [15]:
def clean(text):
    if pd.isna(text): return ""
    return text.replace("\n", " ").strip()

df_filtered.loc[:, "clean_headline"] = df_filtered["headline"].apply(clean)
daily_news = df_filtered.groupby(df_filtered["date"].dt.date)["clean_headline"] \
                        .apply(lambda x: " ".join(x)).reset_index()
daily_news.columns = ["date", "news_text"]

/var/folders/z4/kz3vwpx566d6n7hwbp9czp180000gn/T/ipykernel_25388/1916408444.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, "clean_headline"] = df_filtered["headline"].apply(clean)


In [16]:
os.makedirs("data/news/processed", exist_ok=True)
daily_news.to_csv("data/news/processed/nyt_daily_filtered.csv", index=False)